In [3]:
from flask import Flask, request, jsonify
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

app = Flask(__name__)

@app.route('/')
def index():
    return "Your Flask Python is Running..."

# Load the trained model
model = load_model('Dataset/fashion_mnist_model_improved.h5')

# Define list of class labels
class_labels = ["Kaos Atas", "Celana Panjang", "Sweater", "Gaun", "Mantel",
                "Sandal", "Kemeja", "Sneaker", "Tas", "Sepatu Boot"]

def remove_background(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask for the background
    mask = np.zeros_like(gray)

    # Fill the contours of the background with white color
    cv2.drawContours(mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

    # Invert the mask
    mask = cv2.bitwise_not(mask)

    # Apply the mask to the original image
    result = cv2.bitwise_and(image, image, mask=mask)

    return result

def predict_class(image):
    # Remove background from the image
    processed_image = remove_background(image)
    
    # Resize image to match model input shape
    processed_image = cv2.resize(processed_image, (28, 28))
    
    # Convert image to grayscale
    processed_image = cv2.cvtColor(processed_image, cv2.COLOR_BGR2GRAY)
    
    # Convert image to array and normalize
    processed_image = processed_image.astype('float32') / 255.0
    
    # Expand dimensions to match model input shape
    processed_image = np.expand_dims(processed_image, axis=0)
    processed_image = np.expand_dims(processed_image, axis=3)
    
    # Make prediction using the model
    prediction = model.predict(processed_image)
    predicted_class_index = np.argmax(prediction)
    predicted_class_label = class_labels[predicted_class_index]

    return predicted_class_label

@app.route('/predict', methods=['POST'])
def predict_from_image():
    return jsonify({'error': 'No file part'})
    # if 'file' not in request.files:
    #     return jsonify({'error': 'No file part'})
    # file = request.files['file']
    # if file.filename == '':
    #     return jsonify({'error': 'No file selected'})
    # if file:
    #     # Menerima gambar dari request
    #     file = request.files['file']
    #     nparr = np.frombuffer(file.read(), np.uint8)
    #     image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    #     # Melakukan prediksi kelas
    #     predicted_class = predict_class(image)
        
    #     return jsonify({'predicted_class': predicted_class})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.18.14:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:192.168.18.14 - - [03/Mar/2024 21:26:18] "GET / HTTP/1.1" 200 -
INFO:werkzeug:192.168.18.14 - - [03/Mar/2024 21:26:19] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:192.168.18.14 - - [03/Mar/2024 21:28:10] "GET /predict HTTP/1.1" 405 -
INFO:werkzeug:192.168.18.14 - - [03/Mar/2024 21:28:43] "POST /predict HTTP/1.1" 200 -
